In [ ]:
# Define model features (excluding target and identifiers)
model_features = [
    # Loan characteristics
    'loan_amnt', 'int_rate', 'installment',

    # Borrower financials
    'annual_inc', 'annual_inc_log1p', 'dti',

    # Credit bureau data
    'fico_score', 'delinq_2yrs', 'inq_last_6mths',
    'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
    'total_acc',

    # Engineered features
    'credit_history_years', 'has_delinquency',
    'has_public_records', 'has_bankruptcy',

    # Categorial variables
    'term', 'pymnt_plan', 'application_type', 'disbursement_method',
    'grade', 'sub_grade', 'home_ownership', 'purpose', 'fico_band'

]

data_leakage_vars = ['last_pymnt_amnt', 'last_pymnt_d', 'last_credit_pull_d', 'debt_settlement_flag', 'hardship_flag', 'log_last_pymnt_amnt']
print(f"\nSelected {len(model_features)} features for modeling:")
for i, feat in enumerate(model_features, 1):
    print(f"  {i:2d}. {feat}")

In [ ]:
y = df['default']
X = df[model_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# List of columns to
woe_cols = ['grade', 'sub_grade', 'home_ownership', 'purpose', 'fico_band']
ohe_cols = ['term', 'pymnt_plan', 'application_type', 'disbursement_method']

# WOE Encoder (will replace text in 'woe_cols' with numbers)
woe_encoder = ce.WOEEncoder(cols=woe_cols)

# OneHot Encoder (will replace text in 'ohe_cols' with 0/1 columns)
ohe_encoder = ce.OneHotEncoder(cols=ohe_cols, use_cat_names=True)

# Apply WOE (Pass the WHOLE X_train)
X_train_step1 = woe_encoder.fit_transform(X_train, y_train)
X_test_step1 = woe_encoder.transform(X_test)

# Apply One-Hot to the result of Step A
X_train_step2 = ohe_encoder.fit_transform(X_train_step1)
X_test_step2 = ohe_encoder.transform(X_test_step1)

# Standard scaling for the sake of the Logistic Regression
scaler = StandardScaler()
X_train_final = scaler.fit_transform(X_train_step2)
X_test_final = scaler.transform(X_test_step2)